In [1]:
import cv2
import time
import numpy as np
import pickle
from keras.preprocessing import image
from matplotlib import pyplot
from PIL import Image
import glob
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split


In [2]:
image_list_SUV = []  ## reading images of sedan from path and storing in this list
image_list_sedan = []  ## reading images of sedan from path and storing in this list

## reading the images from the source and saving it in the list after resizing it to 224,224
 ## image size for input to mobilenet is 224 x 224
for filename in glob.glob('car_data/car_data/SUV/*.jpg'): #assuming gif
    im= cv2.imread(filename)
    image_list_SUV.append(cv2.resize(im,(224,224))) 
for filename in glob.glob('car_data/car_data/sedan/*.jpg'): #assuming gif
    im= cv2.imread(filename)
    image_list_sedan.append(cv2.resize(im,(224,224)))


In [3]:
## Dataset preparation

## The images for sedan and suv are combined and sedan is given label as 0 and SUV is given label as 1.

x = np.array(image_list_sedan[:500])    
x = np.concatenate((x,np.array(image_list_SUV[:500])))
y = np.zeros(len(image_list_sedan[:500]))
y = np.concatenate((y, np.ones(len(image_list_SUV[:500]))))



In [4]:
## Split the data into train and test dataset

x_scaled = x / 255  ## normalizing the images

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.30, shuffle=True)


In [5]:
## This model gives all the layer except the last layer
feature_ext_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_freeze = hub.KerasLayer(
    feature_ext_model, input_shape=(224, 224, 3), trainable=False)

## Combining the base model of MobileNet without the top layer and adding our own layer to classifiy two classes
classes_num = 2  ## classifiy two classes- sedan or SUV
model_classifier = tf.keras.Sequential([pretrained_model_freeze, tf.keras.layers.Dense(classes_num)])

model_classifier.summary()
model_classifier.compile(optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [6]:
model_classifier.fit(x_train, y_train, epochs=10)
model_classifier.evaluate(x_test, y_test)


Epoch 1/10
22/22 [==============================] - 37s 1s/step - loss: 0.6574 - acc: 0.6022
Epoch 2/10
22/22 [==============================] - 22s 1s/step - loss: 0.3683 - acc: 0.8531
Epoch 3/10
22/22 [==============================] - 22s 1s/step - loss: 0.2760 - acc: 0.8910
Epoch 4/10
22/22 [==============================] - 22s 1s/step - loss: 0.2425 - acc: 0.9092
Epoch 5/10
22/22 [==============================] - 22s 1s/step - loss: 0.2319 - acc: 0.9115
Epoch 6/10
22/22 [==============================] - 22s 999ms/step - loss: 0.1945 - acc: 0.9125
Epoch 7/10
22/22 [==============================] - 22s 1s/step - loss: 0.1779 - acc: 0.9384
Epoch 8/10
22/22 [==============================] - 23s 1s/step - loss: 0.1681 - acc: 0.9464
Epoch 9/10
22/22 [==============================] - 23s 1s/step - loss: 0.1621 - acc: 0.9359
Epoch 10/10
10/10 [==============================] - 9s 814ms/step - loss: 0.2061 - acc: 0.9267


[0.20609866082668304, 0.9266666769981384]

In [8]:
model_classifier.evaluate(x_test, y_test)

10/10 [==============================] - 9s 923ms/step - loss: 0.2061 - acc: 0.9267


[0.20609866082668304, 0.9266666769981384]

In [9]:
model_classifier.save('class_model_updated') ## Saving the classifier model to reuse it

INFO:tensorflow:Assets written to: class_model_updated\assets


INFO:tensorflow:Assets written to: class_model_updated\assets
